# Advent of Code - 2025 - Day 10 - Problem 2

https://adventofcode.com/2025/day/10

## Load Source Data

Load source data into `DATA`.

In [82]:
# Read the input file containing turn instructions
with open("data/day10.txt") as f:
    DATA = [line.strip() for line in f]

# DATA

## Define Machine class

Defines the state of a machine.

In [83]:
import itertools
from functools import cache
import re


class Machine:

    def __init__(self, definition: str):

        valid_lights, buttons, joltages = Machine.parse_definition(definition)

        self.valid_lights = valid_lights
        self.buttons = buttons
        self.joltages = joltages

    @staticmethod
    def create_patterns() -> tuple[str, str]:

        ws = r"\s+"
        light_pattern = r"\[([.#]+)\]"
        button_pattern = r"\(([0-9,]+)\)"
        joltage_pattern = r"\{([0-9,]+)\}"

        # Regular expression pattern for the definition string
        line_pattern = (
            f"{light_pattern}((?:{ws}{button_pattern})+){ws}{joltage_pattern}"
        )

        # Regular expression for individual button definitions
        button_pattern = f"(?:{ws}({button_pattern}))"

        return (line_pattern, button_pattern)

    @staticmethod
    def parse_definition(
        definition: str,
    ) -> tuple[list[bool], list[list[int]], list[int]]:

        # Parse the overall definition string. Note that there are two results returned for the buttons due to the nested grouping constructs: the entire
        # button string as well as the last matched button.
        #
        definition_match = re.match(Machine.line_pattern, definition)
        assert definition_match
        buttons = definition_match.group(2)
        button_matches = re.findall(Machine.button_pattern, buttons)

        # Parse and convert the separate machine attributes.
        #
        valid_lights: list[bool] = [light == "#" for light in definition_match.group(1)]
        buttons = [
            list(map(int, button_match[1].split(",")))
            for button_match in button_matches
        ]
        joltages = list(map(int, definition_match.group(4).split(",")))

        return (valid_lights, buttons, joltages)
    
    @cache
    def get_min_presses(self) -> dict[tuple[int, ...], int]:

        result: dict[tuple[int, ...], int] = dict()

        for set_size in range(1, len(self.buttons) + 1):
            for button_set in itertools.combinations(self.buttons, set_size):
                button_count = 0
                joltages = [0] * len(self.joltages)
                for button in button_set:
                    button_count += 1
                    for idx_joltage in button:
                        joltages[idx_joltage] += 1
                key = tuple(joltages)
                # print(f"button_set={button_set}, joltages={joltages}, button_count={button_count}")
                if key in result:
                    result[key] = min(button_count, result[key])
                else:
                    result[key] = button_count

        result[tuple([0] * len(self.joltages))] = 0
        return result

    def get_x(self, joltages: tuple[int, ...], patterns: dict[tuple[int, ...], int], level: int) -> int:

        # print(f"{"  " * level} get_x ({joltages})")

        if all(j == 0 for j in joltages): 
            return 0
        
        result = 999999999
        for pattern, pattern_cost in patterns.items():
            if all(p <= j and p % 2 == j % 2 for p, j in zip(pattern, joltages)):
                # print(f"{"  " * level} trying {pattern} for {pattern_cost}")
                new_goal = tuple((j - p) // 2 for p, j in zip(pattern, joltages))
                subcost = 2 * self.get_x(new_goal, patterns, level + 1)
                # print(f"{"  " * level} subcost = {subcost}")
                result = min(result, pattern_cost + subcost)
                # print(f"{"  " * level} current result = {result}")

        # print(f"{"  " * level} result = {result}")
        return result

    def get_lights(self, button_presses: list[bool]) -> list[bool]:
        result: list[bool] = [False] * len(self.valid_lights)
        for idx_button, button_pressed in enumerate(button_presses):
            if button_pressed:
                button = self.buttons[idx_button]
                for idx_light in button:
                    result[idx_light] = not result[idx_light]

        return result

    def get_all_light_solutions(self, lights: list[bool]) -> list[list[bool]]:

        # print(f"get_all_light_solutions {lights}")

        solutions: list[list[bool]] = []

        for test_button_combination in itertools.product(
            [True, False], repeat=len(self.buttons)
        ):
            test_lights = self.get_lights(list(test_button_combination))
            if test_lights == lights:
                solutions.append(list(test_button_combination))

        return solutions

    def get_best_light_solution(self, lights: list[bool]) -> int:

        all_solutions = self.get_all_light_solutions(lights)
        if len(all_solutions) == 0:
            return 999999999

        all_solutions.sort(key=sum)
        best_solution = all_solutions[0]
        return sum(best_solution)

    def get_best_joltage_solution(self, joltages: list[int], cache: dict[tuple[int, ...], int]) -> int:

        # print(f"get_best_joltage_solution {joltages}")

        key = tuple(joltages)

        if key in cache:
            # print("*")
            return cache[key]

        if sum(joltages) == 0:
            cache[key] = 0
            # print(f"  result = {0}")
            return 0
        
        # if max(joltages) == 1:
        #     result = self.get_best_light_solution([joltage == 1 for joltage in joltages])
        #     cache[key] = result
        #     # print(f"  result = {result}")
        #     return result
        
        odd_joltages = [joltage % 2 == 1 for joltage in joltages]
        if any(odd_joltages):
            # remaining_joltages = [
            #     joltage - 1 if odd_joltages[idx_joltage] else joltage
            #     for idx_joltage, joltage in enumerate(joltages)
            # ]
            # remaining_solution = self.get_best_joltage_solution(remaining_joltages, cache)
            # if remaining_solution == 999999999:
            #     cache[key] = remaining_solution
            #     return remaining_solution

            best_solution = 999999999

            button_combinations = self.get_all_light_solutions(odd_joltages)
            # button_combinations =  itertools.product(
            #     [True, False], repeat=len(self.buttons)
            # )
            for button_combination in button_combinations:
                if any(button_combination):
                    test_joltages = [0] * len(joltages)
                    for idx_button, button in enumerate(button_combination):
                        if button:
                            for idx_joltage in self.buttons[idx_button]:
                                test_joltages[idx_joltage] += 1

                    remaining_joltages = [
                        joltage - test_joltages[idx_joltage]
                        for idx_joltage, joltage in enumerate(joltages)
                    ]
                    if all(j >= 0 for j in remaining_joltages):
                        solution = self.get_best_joltage_solution(remaining_joltages, cache)
                        if solution < 999999999:
                            solution = solution + sum(button_combination)

                        best_solution = min(solution, best_solution)

            cache[key] = best_solution
            return best_solution
            

            # return sum(
            #     self.get_best_light_solution(odd_joltages)
            # ) + self.get_best_joltage_solution(remaining_joltages)

        # even_joltages = [joltage % 2 == 0 for joltage in joltages]
        # if all(even_joltages):
        #     result = 2 * self.get_best_joltage_solution([joltage // 2 for joltage in joltages], cache)
        #     cache[key] = result
        #     print(f"  result = {result}")
        #     return result

        # best_result = 999999999
        # for button in self.buttons:
        #     remaining_joltages = [joltage -1 if idx in button else joltage for idx, joltage in enumerate(joltages)]
        #     if all(joltage >= 0 for joltage in remaining_joltages):
        #         result = sum(button) + self.get_best_joltage_solution(remaining_joltages, cache)
        #         best_result = min(result, best_result)
        # cache[key] = best_result
        # print(f"  result = {best_result}")
        # return best_result


        # # See if any joltages are odd.

        # All joltages are even.
        result =  self.get_best_joltage_solution(
            [joltage // 2 for joltage in joltages], cache
        )
        if result != 999999999:
            result = result * 2
        cache[key] = result
        return result

    line_pattern, button_pattern = create_patterns()

## Solve All Machines

In [84]:
total_presses = 0

for idx, line in enumerate(DATA):
    print(f"============ {idx}")
    m = Machine(line)
    patterns = m.get_min_presses()
    print(m.buttons)
    print(patterns)
    cost = m.get_x(tuple(m.joltages), patterns, 0)
    print(f"cost = {cost}")
    total_presses += cost
    # print(m.buttons)
    # presses = m.get_best_joltage_solution(m.joltages, dict())
    # print(presses)
    # total_presses += presses

print(f"total_presses = {total_presses}")

============ 0
[[0, 1, 5, 6, 7, 8, 9], [4, 5], [1, 2, 3, 5, 6], [0, 3], [8, 9], [0, 3, 5, 6, 7, 8, 9], [0, 1, 4, 6, 7, 9], [1, 2], [5, 8]]
{(1, 1, 0, 0, 0, 1, 1, 1, 1, 1): 1, (0, 0, 0, 0, 1, 1, 0, 0, 0, 0): 1, (0, 1, 1, 1, 0, 1, 1, 0, 0, 0): 1, (1, 0, 0, 1, 0, 0, 0, 0, 0, 0): 1, (0, 0, 0, 0, 0, 0, 0, 0, 1, 1): 1, (1, 0, 0, 1, 0, 1, 1, 1, 1, 1): 1, (1, 1, 0, 0, 1, 0, 1, 1, 0, 1): 1, (0, 1, 1, 0, 0, 0, 0, 0, 0, 0): 1, (0, 0, 0, 0, 0, 1, 0, 0, 1, 0): 1, (1, 1, 0, 0, 1, 2, 1, 1, 1, 1): 2, (1, 2, 1, 1, 0, 2, 2, 1, 1, 1): 2, (2, 1, 0, 1, 0, 1, 1, 1, 1, 1): 2, (1, 1, 0, 0, 0, 1, 1, 1, 2, 2): 2, (2, 1, 0, 1, 0, 2, 2, 2, 2, 2): 2, (2, 2, 0, 0, 1, 1, 2, 2, 1, 2): 2, (1, 2, 1, 0, 0, 1, 1, 1, 1, 1): 2, (1, 1, 0, 0, 0, 2, 1, 1, 2, 1): 2, (0, 1, 1, 1, 1, 2, 1, 0, 0, 0): 2, (1, 0, 0, 1, 1, 1, 0, 0, 0, 0): 2, (0, 0, 0, 0, 1, 1, 0, 0, 1, 1): 2, (1, 0, 0, 1, 1, 2, 1, 1, 1, 1): 2, (1, 1, 0, 0, 2, 1, 1, 1, 0, 1): 2, (0, 1, 1, 0, 1, 1, 0, 0, 0, 0): 2, (0, 0, 0, 0, 1, 2, 0, 0, 1, 0): 2, (1, 1, 1, 2, 0, 1, 1